In [212]:
import numpy as np

from pulp import *

# Input data
epsilon = 0.001
ingredients_x = {
    'Maize': {'DM%': 88, 'CP%': 8, 'CF%': 12, 'Ca%': 0.17, 'P%': 0.55, 'ME Kcal/kg': 3000, 'Lysine %': 0.53, 'Methionine %': 0.29, 'min': 0, 'max': 0, 'cost': 23.189546668256277},
    'Maize bran': {'DM%': 88, 'CP%': 9.4, 'CF%': 13, 'Ca%': 0.04, 'P%': 1.03, 'ME Kcal/kg': 2200, 'Lysine %': 0.18, 'Methionine %': 0.21, 'min': 2, 'max': 0, 'cost': 21.87697106165995},
    'Maize/cob meal': {'DM%': 88, 'CP%': 7, 'CF%': 8, 'Ca%': 0, 'P%': 0.3, 'ME Kcal/kg': 0, 'Lysine %': 0, 'Methionine %': 0, 'min': 0, 'max': 0, 'cost': 12.15409672533318},
    'Rice bran': {'DM%': 88, 'CP%': 13.5, 'CF%': 6.5, 'Ca%': 0.06, 'P%': 1.43, 'ME Kcal/kg': 3000, 'Lysine %': 0.5, 'Methionine %': 0.22, 'min': 2, 'max': 0, 'cost': 19.744125331958898},
    'Cassava meal': {'DM%': 88, 'CP%': 2.8, 'CF%': 4.0, 'Ca%': 0.3, 'P%': 0.05, 'ME Kcal/kg': 3000, 'Lysine %': 0, 'Methionine %': 0, 'min': 1, 'max': 0, 'cost': 15.900467964335567},
    'Molasses': {'DM%': 75, 'CP%': 3.0, 'CF%': 0, 'Ca%': 0.75, 'P%': 0.08, 'ME Kcal/kg': 2330, 'Lysine %': 0, 'Methionine %': 0, 'min': 0, 'max': 0, 'cost': 9.74283188303314},
    'Millet': {'DM%': 88, 'CP%': 10.5, 'CF%': 2.0, 'Ca%': 0.05, 'P%': 0.4, 'ME Kcal/kg': 1392, 'Lysine %': 0.2, 'Methionine %': 0.27, 'min': 1, 'max': 0, 'cost': 10.501038943788405},
    'Sorghum': {'DM%': 88, 'CP%': 9.0, 'CF%': 2.1, 'Ca%': 0.03, 'P%': 0.28, 'ME Kcal/kg': 3250, 'Lysine %': 0.2, 'Methionine %': 0.12, 'min': 1, 'max': 0, 'cost': 21.282265115752498},
    'Fish meal': {'DM%': 88, 'CP%': 60.0, 'CF%': 1.0, 'Ca%': 4.37, 'P%': 2.53, 'ME Kcal/kg': 2310, 'Lysine %': 4.08, 'Methionine %': 3.7, 'min': 0, 'max': 0, 'cost': 95.35340164406743},
    'Blood meal': {'DM%': 92, 'CP%': 72.9, 'CF%': 1.7, 'Ca%': 0.28, 'P%': 0.22, 'ME Kcal/kg': 1177, 'Lysine %': 7.0, 'Methionine %': 0.9, 'min': 0, 'max': 0, 'cost': 83.85814098883452},
    'Cotton seed cake': {'DM%': 88, 'CP%': 40.0, 'CF%': 14, 'Ca%': 0.2, 'P%': 1.2, 'ME Kcal/kg': 968, 'Lysine %': 1.6, 'Methionine %': 0.52, 'min': 0, 'max': 0, 'cost': 48.7539532170818},
    'Soya bean meal': {'DM%': 88, 'CP%': 43.0, 'CF%': 6, 'Ca%': 0.53, 'P%': 0.64, 'ME Kcal/kg': 2800, 'Lysine %': 2.84, 'Methionine %': 0.65, 'min': 0, 'max': 0, 'cost': 79.64411128049252},
    'Limestone': {'DM%': 98, 'CP%': 0, 'CF%': 0, 'Ca%': 38.0, 'P%': 0, 'ME Kcal/kg': 0, 'Lysine %': 0, 'Methionine %': 0, 'min': 0, 'max': 0, 'cost': 14.450768262846697},
    'Oyster shells': {'DM%': 98, 'CP%': 0, 'CF%': 0, 'Ca%': 35.0, 'P%': 0, 'ME Kcal/kg': 0, 'Lysine %': 0, 'Methionine %': 0, 'min': 0, 'max': 0, 'cost': 17.437072492457933},
    'Wheat pollard': {'DM%': 98, 'CP%': 15.0, 'CF%': 0, 'Ca%': 0, 'P%': 0, 'ME Kcal/kg': 0, 'Lysine %': 0.6, 'Methionine %': 0.0, 'min': 0, 'max': 0, 'cost': 38.212601422729875},
    # 'methionine': {'DM%': 0, 'CP%': 0, 'CF%': 0, 'Ca%': 0, 'P%': 0, 'ME Kcal/kg': 0, 'Lysine %': 0, 'Methionine %': 2, 'min': 0, 'max': 0, 'cost': 0},
}

nutrient_requirements = {
    'DM%': {'min': None, 'max': None},
    'CP%': {'min': 18, 'max': None},
    'CF%': {'min': None, 'max': 7.5},
    'Ca%': {'min': 0.9, 'max': None},
    'ME Kcal/kg': {'min': 3000, 'max': None},
    'Lysine %': {'min': 1.1, 'max': None},
    'Methionine %': {'min': 0.6, 'max': None},
    
}



# Convert data into NumPy arrays
costs = np.array([ingredients_x[ingredient]['cost'] for ingredient in ingredients_x])


nutrient_content = np.array([[ingredients_x[ingredient][nutrient] for nutrient in nutrient_requirements] for ingredient in ingredients_x])

# Define the problem
prob = LpProblem("Least_Cost_Feed_Formulation", LpMinimize)


# Define the decision variables (amount of each ingredient)
amounts = LpVariable.dicts("Amount", ingredients_x, lowBound=0, cat='Continuous')


# Define the objective function (cost minimization)
prob += lpSum(costs * [amounts[i] for i in ingredients_x]), "Total_Cost"

# Add nutrient constraints
for idx, nutrient in enumerate(nutrient_requirements):
    min_value = nutrient_requirements[nutrient]['min']
    max_value = nutrient_requirements[nutrient]['max']

    # Min constraint
    if min_value is not None:
        prob += lpSum(nutrient_content[:, idx] * [amounts[i] for i in ingredients_x]) >= min_value, f"{nutrient}_min"

    # Max constraint (if not None)
    if max_value is not None:
        prob += lpSum(nutrient_content[:, idx] * [amounts[i] for i in ingredients_x]) <= max_value, f"{nutrient}_max"


prob += lpSum(amounts[i] for i in ingredients_x) == 1, "Total_Quantity_Constraint"


# Solve the problem
status = prob.solve(PULP_CBC_CMD(msg=False))


# Print the status of the solution
print("Status:", LpStatus[status])

# # Print the optimal amounts of each ingredient for feasible solutions
# if LpStatus[status] == "Optimal":
#     print("\nOptimal Amounts:")
for ingredient in ingredients_x:
    if amounts[ingredient].varValue !=0:
        print(f"{ingredient}: {amounts[ingredient].varValue} kg")



total_nutrient = {}
not_satisfied = {}



# Calculate total nutrient values and check if they meet requirements
for nutrient in nutrient_requirements:
    nutrient_values = np.array([ingredients_x[i][nutrient] * amounts[i].varValue if amounts[i].varValue is not None else 0 for i in ingredients_x])
    total_value = np.sum(nutrient_values)
    # Check if the nutrient values satisfy the requirements
    min_value = nutrient_requirements[nutrient]['min']
    max_value = nutrient_requirements[nutrient]['max']


    if min_value is not None and max_value is not None:
        if min_value <= total_value <= max_value:
            total_nutrient[nutrient] = round(total_value, 4)
        else:
            not_satisfied[nutrient] = total_value
    elif min_value is not None:
        if min_value <= total_value:
            total_nutrient[nutrient] = round(total_value, 4)
        else:
            not_satisfied[nutrient] = total_value
    elif max_value is not None:
        if total_value <= max_value:
            total_nutrient[nutrient] = round(total_value, 4)
        else:
            not_satisfied[nutrient] = total_value


# ...


# Print the not satisfied nutrients if any
if not_satisfied:
    print("\nNot Satisfied Nutrients:")
    for nutrient, value in not_satisfied.items():
        print(f"{nutrient}: {value}")

# Print the context with total nutrient values
context = {
    'total_nutrient': total_nutrient,
    'not_satisfied': not_satisfied,
    'status': status,
}

print("\nContext:")
print(context)
print("\nNutrient Requirements:")
print(nutrient_requirements)



Status: Optimal
Sorghum: 0.79449625 kg
Fish meal: 0.15988391 kg
Blood meal: 0.041253485 kg
Limestone: 0.0043663538 kg

Not Satisfied Nutrients:
Ca%: 0.8999999943999999
ME Kcal/kg: 2999.9999964450003
Lysine %: 1.0999999978

Context:
{'total_nutrient': {'CP%': 19.7509, 'CF%': 1.8985, 'Methionine %': 0.724}, 'not_satisfied': {'Ca%': 0.8999999943999999, 'ME Kcal/kg': 2999.9999964450003, 'Lysine %': 1.0999999978}, 'status': 1}

Nutrient Requirements:
{'DM%': {'min': None, 'max': None}, 'CP%': {'min': 18, 'max': None}, 'CF%': {'min': None, 'max': 7.5}, 'Ca%': {'min': 0.9, 'max': None}, 'ME Kcal/kg': {'min': 3000, 'max': None}, 'Lysine %': {'min': 1.1, 'max': None}, 'Methionine %': {'min': 0.6, 'max': None}}


In [213]:
# if ingridient_requirement['min_No_of_ingridients'] is not None:
#     # Add constraint for minimum number of ingredients
#     prob += lpSum(selected_ingredients[i] for i in ingredients_x) >= ingridient_requirement['min_No_of_ingridients'], "Min_Ingredients_Constraint"

#     # Link the binary variables to the continuous variables
#     for ingredient in ingredients_x:
#         prob += amounts[ingredient] <= selected_ingredients[ingredient] * 1e5, f"Link_Less_{ingredient}"
#         prob += amounts[ingredient] >= selected_ingredients[ingredient], f"Link_Greater_{ingredient}"
# Add the total quantity constraint


# Calculate total nutrient values and check if they meet requirements
# for nutrient in nutrient_requirements:
#     nutrient_values = np.array([ingredients_x[i][nutrient] * amounts[i].varValue if amounts[i].varValue is not None else 0 for i in ingredients_x])
#     total_value = np.sum(nutrient_values)
#     # Check if the nutrient values satisfy the requirements
#     min_value = nutrient_requirements[nutrient]['min']
#     max_value = nutrient_requirements[nutrient]['max']


#     if min_value is not None and max_value is not None:
#         if min_value <= total_value <= max_value:
#             total_nutrient[nutrient] = round(total_value, 4)
#         else:
#             not_satisfied[nutrient] = total_value
#     elif min_value is not None:
#         if min_value <= total_value:
#             total_nutrient[nutrient] = round(total_value, 4)
#         else:
#             not_satisfied[nutrient] = total_value
#     elif max_value is not None:
#         if total_value <= max_value:
#             total_nutrient[nutrient] = round(total_value, 4)
#         else:
#             not_satisfied[nutrient] = total_value

In [214]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Input data

# ... (Your input data here)

# Convert data into NumPy arrays
costs = np.array([ingredients_x[ingredient]['cost'] for ingredient in ingredients_x])
nutrient_content = np.array([[ingredients_x[ingredient][nutrient] for nutrient in nutrient_requirements] for ingredient in ingredients_x])

# Create a new model
model = gp.Model("Least_Cost_Feed_Formulation")

# Decision variables
amounts = model.addVars(ingredients_x.keys(), lb=0, vtype=GRB.CONTINUOUS, name="Amount")

# Objective function
model.setObjective(gp.quicksum(costs[i] * amounts[ingredient] for i, ingredient in enumerate(ingredients_x)), GRB.MINIMIZE)

model.addConstr(gp.quicksum(amounts[i] for i in ingredients_x) == 1, "Total_Quantity_Constraint")

# Nutrient constraints
for idx, nutrient in enumerate(nutrient_requirements):
    min_value = nutrient_requirements[nutrient]['min']
    max_value = nutrient_requirements[nutrient]['max']

    # Use the same variable for both lb and ub to create equality constraint
    model.addConstr(gp.quicksum(nutrient_content[:, idx] * amounts[ingredient] for ingredient in ingredients_x) == min_value, f"{nutrient}_min")

    if max_value is not None:
        model.addConstr(gp.quicksum(nutrient_content[:, idx] * amounts[ingredient] for ingredient in ingredients_x) == max_value, f"{nutrient}_max")

# Ingredient amount constraints
for ingredient in ingredients_x:
    model.addConstr(amounts[ingredient] >= ingredients_x[ingredient]['min'], f"{ingredient}_min")

    if ingredients_x[ingredient]['max'] != 0:
        model.addConstr(amounts[ingredient] <= ingredients_x[ingredient]['max'], f"{ingredient}_max")

# Total quantity constraint
# model.addConstr(gp.quicksum(amounts[i] for i in ingredients_x) == 1, "Total_Quantity_Constraint")

# Solve the model
model.optimize()

# Print the status of the solution
print("Status:", model.status)

# Print the optimal amounts of each ingredient for feasible solutions
if model.status == GRB.OPTIMAL:
    print("\nOptimal Amounts:")
    for ingredient in ingredients_x:
        print(f"{ingredient}: {amounts[ingredient].x} kg")

    # Print the achieved nutrient values for feasible solutions
    print("\nAchieved Nutrient Values:")
    for nutrient in nutrient_requirements:
        achieved_value = np.sum([nutrient_content[i, idx] * amounts[ingredient].x for i, ingredient in enumerate(ingredients_x)])
        print(f"{nutrient}: {achieved_value}")

    # Print the total cost of the feed mixture for feasible solutions
    print("\nTotal Cost:", round(np.sum(costs * [amounts[ingredient].x for ingredient in ingredients_x]), 2))

# Print information for infeasible solutions
else:
    print("\nThe problem is infeasible or unbounded.")
    # Print infeasible variable values
    print("\nInfeasible Variable Values:")
    for ingredient in ingredients_x:
        print(f"{ingredient}: {amounts[ingredient].index} kg")


TypeError: unsupported operand type(s) for -: 'bool' and 'NoneType'

In [ ]:
# Define a function named decode that takes a message file as an argument
def decode(message_file):
  # Open the message file for reading
  with open(message_file, "r") as f:
    # Initialize an empty dictionary to store the number-word pairs
    num_word = {}
    # Loop through each line in the file
    for line in f:
      # Split the line by whitespace and get the first and second elements
      num, word = line.split()
      # Convert the first element to an integer and use it as a key in the dictionary
      # Use the second element as the value in the dictionary
      num_word[int(num)] = word
    # Initialize an empty string to store the decoded message
    message = ""
    # Initialize a variable to store the current row number
    row = 1
    # Loop until the row number exceeds the maximum key in the dictionary
    while row <= max(num_word.keys()):
      # Check if the row number is a key in the dictionary
      if row in num_word:
        # If yes, add the corresponding word to the message, followed by a space
        message += num_word[row] + " "
      # Increment the row number by 1
      row += 1
    # Return the message, stripping any trailing whitespace
    return message.strip()
decode('a.txt')

'I dogs love cats you computers'

In [ ]:
# Define a function named decode that takes a message file as an argument
def Decode(message_file):
  with open(message_file, "r") as f:
    num_word = {int(num): word for num, word in (line.split() for line in f)}
    values_list = [word for _, word in sorted(num_word.items())]
  # Initialize an empty string to store the decoded message
  message_words = []
  # Initialize a variable to store the current row number
  pyramid_count = 0

  for index, _ in enumerate(range(len(values_list)), start=1):
      pyramid_count += index
      if pyramid_count - 1 < len(values_list):
          message_words.append(values_list[pyramid_count - 1])
         

  message = ' '.join(message_words)

  return message
result= Decode('a.txt')
print(result)



opposite sun rain think ocean to winter wild it ready buy card possible would electric stay post paragraph produce state our compare touch possible


In [ ]:
def decode_pyramid(message_file):
  with open(message_file, "r") as f:
    num_word = {int(num): word for num, word in (line.split() for line in f)}
    values_list = [word for _, word in sorted(num_word.items())]
    print(values_list)
    
    message = " ".join(word for i, word in enumerate(values_list, start=1) if i * (i + 1) // 2 <= len(values_list))
    return message
decode_pyramid('a.txt')

['I', 'dogs', 'love', 'cats', 'you', 'computers']


'I dogs love'

In [ ]:
values_list = ['I', 'dogs', 'love', 'cats', 'you', 'computers']
message_words = []
pyramid_count = 0

for index, _ in enumerate(range(len(values_list)), start=1):
    pyramid_count += index
    if pyramid_count - 1 < len(values_list):
        message_words.append(values_list[pyramid_count - 1])

message = ' '.join(message_words)
print(message)


I love computers


In [ ]:
values_list = ['I', 'dogs', 'love', 'cats', 'you', 'computers']
message = " ".join(word for index, word in enumerate(values_list, start=1) if index * (index + 1) // 2 <= len(values_list))
print(message)

I dogs love
